In [92]:
import pandas as pd
from rdflib import Graph, Literal, RDF, URIRef, Namespace
from rdflib.namespace import RDFS
import urllib.parse

In [29]:
headers = ['Област', 'Мъже 2016', 'Жени 2016', 'Мъже 2017', 'Жени 2017', 'Мъже 2018', 'Жени 2018', 'Мъже 2019', 'Жени 2019', 'Мъже 2020', 'Жени 2020', 'Мъже 2021', 'Жени 2021']

In [40]:
born = pd.read_csv('born.csv', sep=';', skiprows=3, header=None).drop(13, axis=1)
born.columns = ['Област', 'Родени момчета 2016', 'Родени момичета 2016', 'Родени момчета 2017', 'Родени момичета 2017', 'Родени момчета 2018', 'Родени момичета 2018', 'Родени момчета 2019', 'Родени момичета 2019', 'Родени момчета 2020', 'Родени момичета 2020', 'Родени момчета 2021', 'Родени момичета 2021']
print(born.shape)
born.head()

(28, 13)


,Област,Родени момчета 2016,Родени момичета 2016,Родени момчета 2017,Родени момичета 2017,Родени момчета 2018,Родени момичета 2018,Родени момчета 2019,Родени момичета 2019,Родени момчета 2020,Родени момичета 2020,Родени момчета 2021,Родени момичета 2021
0,Видин,293,263,288,284,304,282,262,286,242,231,281,247
1,Враца,761,692,677,643,701,674,669,679,678,656,633,609
2,Ловеч,507,457,533,476,505,523,506,463,482,500,496,469
3,Монтана,571,535,508,511,500,504,486,450,459,464,493,460
4,Плевен,1111,1071,1089,1023,957,988,1009,993,988,935,892,894


In [41]:
died = pd.read_csv('died.csv', sep=';', skiprows=3, header=None).drop(13, axis=1)
died.columns = ['Област', 'Умрели мъже 2016', 'Умрели жени 2016', 'Умрели мъже 2017', 'Умрели жени 2017', 'Умрели мъже 2018', 'Умрели жени 2018', 'Умрели мъже 2019', 'Умрели жени 2019', 'Умрели мъже 2020', 'Умрели жени 2020', 'Умрели мъже 2021', 'Умрели жени 2021']
print(died.shape)
died.head()

(28, 13)


,Област,Умрели мъже 2016,Умрели жени 2016,Умрели мъже 2017,Умрели жени 2017,Умрели мъже 2018,Умрели жени 2018,Умрели мъже 2019,Умрели жени 2019,Умрели мъже 2020,Умрели жени 2020,Умрели мъже 2021,Умрели жени 2021
0,Видин,1043,1038,1018,981,1029,963,957,965,1173,1096,1385,1197
1,Враца,1659,1545,1704,1655,1659,1582,1697,1483,1906,1633,2301,2063
2,Ловеч,1315,1195,1408,1291,1312,1170,1354,1228,1441,1313,1721,1576
3,Монтана,1542,1354,1448,1364,1425,1416,1400,1399,1754,1462,1991,1824
4,Плевен,2403,2221,2333,2293,2393,2174,2405,2219,2701,2399,3192,2954


In [42]:
total = pd.read_csv('population.csv', sep=';', skiprows=3, header=None).drop(9, axis=1).drop(1, axis=1)
total.columns = ['Област', 'Пол', 'Брой 2016', 'Брой 2017', 'Брой 2018', 'Брой 2019', 'Брой 2020', 'Брой 2021']
print(total.shape)
total.head()

(56, 8)


,Област,Пол,Брой 2016,Брой 2017,Брой 2018,Брой 2019,Брой 2020,Брой 2021
0,Видин,Мъже,43400,42411,41349,40369,39487,38216
1,Видин,Жени,45467,44516,43516,42466,41725,40598
2,Враца,Мъже,83036,81459,79894,78234,77196,75100
3,Враца,Жени,85691,84186,82655,81236,80441,78600
4,Ловеч,Мъже,63102,61906,60836,59658,59580,58181


In [95]:
regions = born['Област']
years = list(range(2016, 2022))

combined = pd.DataFrame()

combined_columns = ['oбласт', 'година', 'родениМомчета', 'родениМомичета', 'умрелиМъже', 'умрелиЖени', 'общоМъже', 'общоЖени']
for region in regions:
    for year in years:
        row_to_add = {
            combined_columns[0]: region,
            combined_columns[1]: year,
            combined_columns[2]: born[born['Област'] == region][f'Родени момчета {year}'].values[0],
            combined_columns[3]: born[born['Област'] == region][f'Родени момичета {year}'].values[0],
            combined_columns[4]: died[died['Област'] == region][f'Умрели мъже {year}'].values[0],
            combined_columns[5]: died[died['Област'] == region][f'Умрели жени {year}'].values[0],
            combined_columns[6]: total[(total['Област'] == region) & (total['Пол'] == 'Мъже')][f'Брой {year}'].values[0],
            combined_columns[7]: total[(total['Област'] == region) & (total['Пол'] == 'Жени')][f'Брой {year}'].values[0]
        }

        combined = combined.append(row_to_add, ignore_index=True)

combined.head()

/tmp/ipykernel_328276/3372793346.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined = combined.append(row_to_add, ignore_index=True)
/tmp/ipykernel_328276/3372793346.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined = combined.append(row_to_add, ignore_index=True)
/tmp/ipykernel_328276/3372793346.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined = combined.append(row_to_add, ignore_index=True)
/tmp/ipykernel_328276/3372793346.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined = combined.append(row_to_add, ignore_index=True)
/tmp/ipykernel_328276/3372793346.py:20: FutureWarning: The frame.app

,Област,Година,Родени момчета,Родени момичета,Умрели мъже,Умрели жени,Общо мъже,Общо жени
0,Видин,2016,293,263,1043,1038,43400,45467
1,Видин,2017,288,284,1018,981,42411,44516
2,Видин,2018,304,282,1029,963,41349,43516
3,Видин,2019,262,286,957,965,40369,42466
4,Видин,2020,242,231,1173,1096,39487,41725


In [93]:
g = Graph()

# RDF and RDFS are already imported
cube = Namespace('https://cube.link/')
schema = Namespace('http://schema.org/')